In [100]:
import os

In [101]:
%pwd

'c:\\Users\\Public\\mycode\\PROJECT-1'

In [102]:
os.chdir("c:\\Users\\Public\\mycode\\PROJECT-1") # Changing my directory to project-1

In [103]:
%pwd

'c:\\Users\\Public\\mycode\\PROJECT-1'

In [104]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataIngestionconfig:
    root_dir:Path
    source_url:str
    local_data_file:Path
    unzip_dir:Path

In [105]:
from src.datascience.constants import *

In [106]:
from src.datascience.utils.common import read_yaml, create_directories

In [ ]:
class ConfigurationManager:
    def __init__(self,
                 config_file_path = CONFIG_FILE_PATH,
                 params_file_path = PARAMS_FILE_PATH,
                 schema_file_path = SCHEMA_FILE_PATH):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        self.schema = read_yaml(schema_file_path)
        
        create_directories([self.config.artifacts_root])
    
    def get_data_ingestion_config(self) -> DataIngestionconfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])
        
        data_ingestion_config = DataIngestionconfig(
            root_dir = config.root_dir,
            source_url = config.source_url, 
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir
        )
        return data_ingestion_config
        

In [108]:
import os
import urllib.request as request
from src.datascience import logger
import zipfile

In [109]:
class DataIngestion:
    def __init__(self, config: "DataIngestionconfig"):  # Assuming you have a config class
        self.config = config
        
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url=self.config.source_url,
                filename=self.config.local_data_file
            )
            logger.info(f"{filename} downloaded! with following info:\n{headers}")
        else:
            logger.info(f"File already exists at {self.config.local_data_file}")
    
    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
            logger.info(f"Extracted {self.config.local_data_file} to {unzip_path}")

In [110]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-10-02 15:51:22,314:INFO:common:yaml file: config\config.yaml loaded successfully]
[2025-10-02 15:51:22,316:INFO:common:yaml file: params.yaml loaded successfully]
[2025-10-02 15:51:22,318:INFO:common:yaml file: schema.yaml loaded successfully]
[2025-10-02 15:51:22,319:INFO:common:created directory at: artifacts]
[2025-10-02 15:51:22,321:INFO:common:created directory at: artifacts/data_ingestion]
[2025-10-02 15:51:23,532:INFO:2396004329:artifacts/data_ingestion/data.zip downloaded! with following info:
Connection: close
Content-Length: 23329
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "c69888a4ae59bc5a893392785a938ccd4937981c06ba8a9d6a21aa52b4ab5b6e"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 2C98:54918:AF6D0:1BE07F:68DE521E
Accept-Ranges: bytes
Date: Thu, 02 Oct 2025 10:21:22 